In [7]:
import numpy as np
import torch

In [1]:
import pyro
assert pyro.__version__.startswith('1.8.3')
pyro.set_rng_seed(1)

In [2]:
loc = 0.   # mean zero
scale = 1.  # unit stddev
# create a normal distribution object
normal = pyro.distributions.Normal(loc, scale)
# draw a sample from N(0,1)
x = normal.sample()
print("sample", x)
# score the sample from N(0,1)
print("log prob", normal.log_prob(x))


sample tensor(0.6614)
log prob tensor(-1.1376)


In [9]:
x.shape

torch.Size([])

In [8]:
torch.exp(normal.log_prob(x))


tensor(0.3206)

In [3]:
def weather():
    cloudy = pyro.distributions.Bernoulli(0.7).sample()
    cloudy = 'cloudy' if cloudy.item() == 1 else 'sunny'
    mean_temp = {'cloudy': 10.0, 'sunny': 11.0}[cloudy]
    scale_temp = {'cloudy': 1.0, 'sunny': 2.0}[cloudy]
    temp = pyro.distributions.Normal(mean_temp, scale_temp).sample()
    return cloudy, temp.item()


In [4]:
def weather2():
    cloudy = pyro.sample('cloudy',
                         pyro.distributions.Bernoulli(0.7))
    cloudy = 'cloudy' if cloudy.item() == 1.0 else 'sunny'
    mean_temp = {'cloudy': 10.0, 'sunny': 11.0}[cloudy]
    scale_temp = {'cloudy': 1.0, 'sunny': 2.0}[cloudy]
    temp = pyro.sample('temp',
                       pyro.distributions.Normal(mean_temp, scale_temp))
    return cloudy, temp.item()

In [5]:
def ice_cream_sales():
    cloudy, temp = weather()
    expected_sales = 200. if cloudy == 'sunny' and temp > 80.0 else 50.
    ice_cream = pyro.sample('ice_cream',
                            pyro.distributions.Normal(expected_sales, 10.0))
    return ice_cream


In [10]:
d = pyro.distributions.Bernoulli(
    torch.tensor([0.5, 0.5]))
x = d.sample()
assert x.shape == d.batch_shape + d.event_shape
assert d.batch_shape == (2,)
assert d.event_shape == ()
assert d.log_prob(x).shape == d.batch_shape


In [13]:
d.log_prob(x)

tensor([-0.6931, -0.6931])

In [19]:
parameters = torch.tensor([[0.0, 0.0, 0.0], [1.0, 1.0, 1.0]])
d = pyro.distributions.Normal(loc=parameters[0], scale=parameters[1])
x = d.sample()


In [20]:
d.event_shape

torch.Size([])

In [26]:
a = torch.tensor([10,20])

In [25]:
b = torch.tensor([0,1,1,1,0,1,0,1])

In [27]:
a[b]

tensor([10, 20, 20, 20, 10, 20, 10, 20])

In [23]:
with pyro.plate("x_plate", 10):
    # Note that .expand([10]) is automatic
    x = pyro.sample("x", pyro.distributions.Normal(0, 1))
assert x.shape == (10,)
x.batch_shape

AttributeError: 'Tensor' object has no attribute 'batch_shape'